In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#read the csv file 
df = pd.read_csv('https://raw.githubusercontent.com/RussellDash332/CS3244-Twemoji/main/Datasets/train_text_emoji_clean.csv',sep=",",header=0)

In [ ]:
#the dataset
df

,annotations,tweets,labels
0,😂,school is so dead o my god,[0]
1,😂,lol I've been told mine is worse than yours bu...,[0]
2,😂,I'm excited to hear them..... That shit is goi...,[0]
3,😂,Damn alycia knows everything even indirect tweets,[0]
4,😂,x_juicebox: That sound like everything,[0]
...,...,...,...
59902,🙃,FinesseYoAss: Or That Picture Was Just Cute To...,[11]
59903,🙃,I'm otw wit some gas still hate me,[11]
59904,🙃,Yeahhh you kinda do,[11]
59905,🙃,nothing wit youuuuu raymundooo,[11]


In [ ]:
#convert labels to array

def convert(labels):
    lst = [0 for i in range(12)]
    my_string = labels[1:-1]
    result = [x.strip() for x in my_string.split(',')]
    
    for i in result:
        lst[int(i)] = 1
    return lst
df["label"] = df["labels"].apply(convert)

In [ ]:
df

,annotations,tweets,labels,label
0,😂,school is so dead o my god,[0],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,😂,lol I've been told mine is worse than yours bu...,[0],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,😂,I'm excited to hear them..... That shit is goi...,[0],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,😂,Damn alycia knows everything even indirect tweets,[0],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,😂,x_juicebox: That sound like everything,[0],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...
59902,🙃,FinesseYoAss: Or That Picture Was Just Cute To...,[11],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
59903,🙃,I'm otw wit some gas still hate me,[11],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
59904,🙃,Yeahhh you kinda do,[11],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
59905,🙃,nothing wit youuuuu raymundooo,[11],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"


In [ ]:
df = df.dropna()

In [ ]:
df

,annotations,tweets,labels,label
0,😂,school is so dead o my god,[0],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,😂,lol I've been told mine is worse than yours bu...,[0],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,😂,I'm excited to hear them..... That shit is goi...,[0],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,😂,Damn alycia knows everything even indirect tweets,[0],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,😂,x_juicebox: That sound like everything,[0],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...
59902,🙃,FinesseYoAss: Or That Picture Was Just Cute To...,[11],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
59903,🙃,I'm otw wit some gas still hate me,[11],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
59904,🙃,Yeahhh you kinda do,[11],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
59905,🙃,nothing wit youuuuu raymundooo,[11],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"


 0    1186
 1     561
-1     253
Name: sentiment, dtype: int64

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#tokenizing the first 2000 sentences
tokenized = df["tweets"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
#making all the vectors the same size to pass through(aka same as the max size)
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
#checking the size of all the padding 
np.array(padded).shape

(59905, 55)

In [ ]:
#ignore the padding used
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(59905, 55)

In [ ]:
#run the model of distibert
#this part gets jammed
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
#slicing the first token and taking it as the features
features = last_hidden_states[0][:,0,:].numpy()